In [ ]:
from mikecloudio import wrang, request

## Credentials

In [ ]:
key = ""

In [2]:
# get project ID
con = request.ConnectMikeCloud(api_key=key)
id_proj = con.query_proj_id(proj_name="Stadt Geretsried")

# 1) Establish connection to MikeCloud

In [ ]:
# create instance of ConnectMikeCloud:
con = request.ConnectMikeCloud(api_key=key,id_proj=id_proj)


In [ ]:
# Alternatively: 
con = request.ConnectMikeCloud(api_key=key)

proj = con.set_project(id_proj=id_proj)
# -> returns directly the project object


## ---> Functions for ConnectMikeCloud Objects:

In [ ]:
# get all projects:
df = con.list_projects()
df

In [ ]:
# get project ID by name:
con.query_proj_id(proj_name="Stadt Geretsried")

In [ ]:
# get project ID
con._id_proj
# OR
proj._id

In [ ]:
# list datasets
con.list_ds()

In [ ]:
# get dataset ID by name
ds_id = con.query_ds_id(ds_name="test")

In [ ]:
# create dataset and its instance (optional variables: properties, metadata, content_type)
ds = con.create_ds(ds_name="Geretsried TS",ds_description="Dataset für Geretsried Messstellen")

In [ ]:
# delete dataset
con.del_ds(id_ds=ds._id)

In [ ]:
# update dataset (does not work yet)
con.update_ds(dataset_id=ds_id, upd_name="updated_dataset", upd_descr="updated description")

In [ ]:
# get dataset (returns an instance of Dataset)
ds = con.get_ds(id_dataset="")
# OR
ds = con.get_ds(name_ds="Downstream_tide.dfs0")

# 2) Create Dataset Object

In [ ]:
# Option 1: 
ds = request.Dataset(connection=con,dataset_id="")

# Option 2:
# ds = con.get_ds(id_dataset="")
# OR with name
ds = con.get_ds(name_ds="Geretsried TS")

## ---> Functions for Dataset Objects:

In [ ]:
ds._id

In [ ]:
# get details of dataset
ds.get_ds_info()

In [ ]:
# if ID of dataset not set in constructor:
ds.set_ds_id(dataset_id="")

In [ ]:
# get all timeseries
ds.list_ts()

In [ ]:
# query timeseries id from all timeseries names
ds.query_ts_id("MP05")

In [ ]:
# creates a timeseries and returns an instances of the newly created timeseries 
ts = ds.create_ts(name="MP05", unit="eumUmeter", item="eumIWaterLevel", data_type="Single", data_fields=None,
                  properties=None)

ts._id

In [ ]:
# delete dataset (corresponding to the instance id)
ds.del_ds()

# delete timeseries
ds.del_ts(timeseries_id=ts._id)

# 3) Create Timeseries Object

In [ ]:
# Option 1: 
#ts = request.Timeseries(dataset=ds,timeseries_id="")

# Option 2: from dataset object
ts = ds.get_ts(ts_name="MP05")
#OR
#ts = get_ts(ts_id="")

## ---> Functions for Timeseries Objects:

In [ ]:
# get data of timeseries with set from/to time or without: 
''' 
If {from} is not specified, values from the first time step are returned. 
If {to} is not specified, values to the last time step are returned.
'''
time_from = "2018-03-08T150000"
time_to = "2018-03-08T200000"

ts.get_data(time_from=time_from,time_to=time_to)

In [ ]:
# add data in form of a dataframe to timeseries (df needs columns with name "timestamp" and "value")
add_data(dataframe=df)

## Shortcut 

In [ ]:
# method chaining:
con = request.ConnectMikeCloud(api_key='7db3e60a-ea79-4c3c-af0a-06fdcbd6cf90',id_proj='87b3a13d-9f71-4dcc-8843-26a37ff6b49f')
ts = con.get_ds(name_ds="Geretsried TS").get_ts(ts_name="MP05")
# OR shorter:
ts = con.get_ds("Geretsried TS").get_ts("MP05")

In [ ]:
ts.get_data()

## Direct POST from Insitu Troll API to MIKE CLOUD

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta, timezone, date
from oauthlib.oauth2 import BackendApplicationClient
from requests.auth import HTTPBasicAuth
from requests_oauthlib import OAuth2Session


In [ ]:
# need to define client secret (api key) for insitu before 
client_secret = ""

In [ ]:
forward_data(start_time="2018-03-08 13:30:00",api_key=con._api_key,project_id=con._id_proj,dataset_id=ds._id,timeseries_id=ts._id,metadata_url=con.metadata_service_url,client_secret=client_secret)

In [ ]:
def forward_data(start_time,api_key,project_id,timeseries_id,dataset_id,metadata_url,client_secret):
    
    # authorization
    base_url = 'https://www.hydrovu.com/public-api/oauth/token'
    client_id = 'DHIhydroAPI'

    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)
    token = oauth.fetch_token(token_url=base_url, client_id=client_id,
            client_secret=client_secret)
    token = token['access_token']

    headers= {'Authorization': 'Bearer %s' %token}

    header_MIKECLOUD = {'dhi-open-api-key': '{0}'.format(api_key),
               'dhi-project-id': '{0}'.format(project_id),
               'dhi-dataset-id': '{0}'.format(dataset_id),
               'dhi-service-id': 'timeseries',
               'Content-Type': 'application/json',
              }
    link = "{0}api/upload/{1}/timeseries/{2}/json".format(metadata_url,dataset_id,timeseries_id)
    datetime_csv = start_time 
    startTime = datetime.strptime(datetime_csv, '%Y-%m-%d %H:%M:%S')
    startTime = datetime.timestamp(startTime)

    startTime = startTime + 9000 #shift time to %H:30:00 for request stability
    endTime = startTime + 18000   #10 hours later
    currentTime = (datetime.now() - datetime(1970,1,1)).total_seconds()
    hr = 3600    #measurements are hourly
    time_hr = (range(int(startTime/hr),int(currentTime/hr),5)) #list of time in 5 hour intervals from last measurement until now

    for i in time_hr:

        url_data = ("https://www.hydrovu.com/public-api/v1/locations/6444867060236288/data?startTime=%d&endTime=%d"% (startTime,endTime))
        a = datetime.utcfromtimestamp(startTime).strftime('%Y-%m-%d %H:%M:%S')
        b = datetime.utcfromtimestamp(endTime).strftime('%Y-%m-%d %H:%M:%S')
        print("StartTime: %s " % a)
        print("EndTime %s" %b)   
            #read data and convert
        response = requests.get(url_data, headers=headers)
        
        response_dict = response.json()
        response_dep = response_dict['parameters'][1]['readings']

        if not response_dep:
            print("no data for this timestamp")
            startTime = startTime + 18000   #update url value startTime
            endTime = startTime + 18000     #update url value endTime
            continue

        #convert to dataframe, adjust unix to realtime
        depth_to_water = pd.DataFrame(response_dep)
        depth_to_water['timestamp'] = pd.to_datetime(depth_to_water['timestamp'],unit='s')  

        timestamp_str_import = depth_to_water["timestamp"]
       
        for index_timestamp in range(len(timestamp_str_import)):
            timestamp_dt = timestamp_str_import[index_timestamp]
            timestamp_iso = datetime.isoformat(timestamp_dt)

            body = '{{"data": [["{0}", {1}]]}}'.format(timestamp_iso,depth_to_water["value"][index_timestamp])
            response = requests.post(link, headers=header_MIKECLOUD, data=body)
            status = response.status_code
        startTime = startTime + 18000   #update url value startTime
        endTime = startTime + 18000     #update url value endTime

    print("done")
    
